#Namespaces in Python

##Namespaces

Every Python module has a ___global___ namespace, where names are bound by assignments and function definitions within the main body of the program. When you call a function or method, Python dynamically creates a new namespace and binds the argument values to the parameter names in that namespace. Assignments made during execution of the function call (normally) result in bindings in the function call  or ___local___) namespace. When the function returns, the namespace is automatically destroyed, and any bindings inside the namespace are lost. If any objects are no longer referenced as a result of this, the memory used to store those values becomes reclaimable as garbage.

When you write large programs "monolithically" (as whole chunks), we can inadvertently use the same name for two different purposes at different places in the program. You can avoid that problem by incorporating the principle of modularity into your work. It is much easier to write programs as collections of small chunks that are relatively independent of one another. This will also make your programs easier to read and understand, so it's a win-win.

Python also constructs a namespace when you create a class, and in general every object (and everything is on object) in Python has a namespace. Python uses what's called a “late binding” technique. This means dynamically resolves names at run-time by looking in a succession of namespaces. The same name can be defined in multiple namespaces, because the uses don't collide.

### Top-Level Names

When the interpreter needs to resolve an unqualified name (which for these purposes is one not preceded by a dot) it looks in three specific namespaces.

 First, it looks in the local namespace (assuming a function call is active - for code at module level the local and global namespaces are the same). Next, it looks in the global namespace (remember, each module has its own global namespace - there is no _program_ global namespace). Finally, it looks in the "built-in" namespace, which holds the names of objects that are hard-wired into the Python interpreter, like exceptions and built-in functions. If the desired name is not found the interpreter raises a `NameError` exception.

Here's a simple demonstration that the local and global namespaces are the same for code running at module level.

In [ ]:
locals() == globals()

Next we see that a function call creates a separate namespace, initially containing the argument values.

In [ ]:
def any_function(a, b, c):
    print(locals())
    return locals() == globals()
print(any_function(1, 2, 3))

The built-in namespace you can think of as the “namespace of last resort.” It contains everything you can use in Python without having to import an external module.

In [ ]:
dir(__builtin__)

### Attribute Name Resolution

When you create a class it too has a namespace, in which any bindings in the class body are recorded. This includes method definitions as well as assignments. Assignments within the body give rise to so-called _class variables_, stored in the class namespace.

In [ ]:
class DemoClass:
    """Not intended to actually do anything much."""
    instance_count = 0
    def __init__(self, thing):
        self.thing = thing
        DemoClass.instance_count += 1
    def check_thing(self, thing):
        return thing == self.thing

In [ ]:
dir(DemoClass)

We'll talk shortly about where all those dunder methods come from.

You'll see that the `DemoClass` class has an attribute called `__dict__`. This is actually where the repository for the names strictly local to that class.

In [ ]:
type(DemoClass.__dict__)

In [ ]:
for (key, value) in DemoClass.__dict__.items():
    print("{}: {!r}".format(key, value))

Instances also have an associated namespace in which _instance variables_ are stored (these are the ones whose names qualify `self` in your methods' code).

In [ ]:
demo_object = DemoClass(42)
dir(demo_object)

The instance's namespace is also accessible through the name `__dict__`. In this case, though, it's just a regular dict.

In [ ]:
type(demo_object.__dict__)

In [ ]:
for key, value in demo_object.__dict__.items():
    print("{}: {!r}".format(key, value))

Again you see that the `__dict__` contains only the instance attributes, but `dir()` reports the availability of many other attributes. 

This is because attribute access (dotted names) is performed rather differently, and `dir()` tries to report all available attributes. To evaluate the expression `a.z` the interpreter first used the three-namespace rule (locals, then globals , then the built-in namespace) to look up the value of `a`.

Assuming that `a` is found and no `NameError` exception is raised, the interpreter then searches `a`’s namespace for the attribute `z`. If it fails to find it, the interpreter then looks in the instance's class's (in this case `DemoClass`'s) namespace. If it fails to find it there then it looks in that class's parent class's namespace, and so on until it arrives at `object`, the ultimate super-class of everything in Python. If the desired name is not found then an `AttributeError` exception is raised.

If multiple levels of qualification are used (as, for example, in `a.b.c.d`) then the process is repeated for each successive level of qualification.

In [ ]:
DemoClass.instance_count

In [ ]:
demo_object.instance_count

In [ ]:
demo_2 = DemoClass(demo_object)
demo_2.instance_count, demo_2.thing.instance_count

###Possible Discussions

* Is there any Python object that doesn't have its own namespace?
* What's the role of lexical scoping?

###And, of course, whatever _you_ want ...